In [61]:
import re
import tinys3
import xml.etree.ElementTree as ET
from urllib import urlopen, urlretrieve, quote
from urlparse import urljoin
from bs4 import BeautifulSoup
import zipfile
import csv
import boto3
import ftplib
import gzip
import io
import zipfile
import os
import time
import string

DATA_PATH = '/Users/patrickkennedy/Desktop/Data_Science/Patently/data/ipg160712.xml'
ZIP_PATH = '/Users/patrickkennedy/Desktop/Data_Science/Patently/code/'
EXTRACT_DATA_PATH = '/Users/patrickkennedy/Desktop/Data_Science/Patently/data/'

################################################################
########### THIS NEXT PART ACTUALLY WORKS BUT IS MESSY #########
################################################################

def open_many(n):

    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    doc_list = []
    counter = 0

    with open(DATA_PATH, 'r') as f:
        doc = []  # (re)initialize current XML doc to empty list
        for line in f:
            doc.append(line)
            endtag = endtag_regex.findall(line) if not endtag else endtag
            if not endtag:
                continue
            terminate = re.compile('^</{0}>'.format(endtag[0]))
            if terminate.findall(line):
                doc_list.append(doc)
                doc = []
                counter += 1
                if counter == n:
                    return doc_list

                # yield (_get_date(filename), doc)
                # endtag = ''
                # doc = ''


def open_all():

    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    doc_list = []
    counter = 0

    with open(DATA_PATH, 'r') as f:
        doc = []  # (re)initialize current XML doc to empty list
        for line in f:
            doc.append(line)
            endtag = endtag_regex.findall(line) if not endtag else endtag
            if not endtag:
                continue
            terminate = re.compile('^</{0}>'.format(endtag[0]))
            if terminate.findall(line):
                doc_list.append(doc)
                doc = []
        return doc_list
                
                # yield (_get_date(filename), doc)
                # endtag = ''
                # doc = ''

In [4]:
# num_patents = 1000
counter = 0
data = open_all()
                
# data = open_all()
#this first toy file has 4046 (i think) files
#let's see how many are junked.... --> 89

In [58]:
for i in xrange(len(data)):
    if i == 1528:
        data_as_string = "".join(data[i])
        try:
            root = ET.fromstring(data_as_string)
        except:
            counter += 1
            root = None

        if root:
            row = write_row(root)
            print row
    if i > 1539:
        break

('09387704', '20160712', 'B2', '', 'Ink-jet recording apparatus', ' There is provided an ink-jet recording apparatus which includes a recording portion, and a contact portion configured to come into contact with an upper surface of the sheet. The contact portion has a storage portion provided on a side of the recording portion and configured to store an ink, and a protruding portion is provided between both ends in a width direction intersecting the transporting direction of the storage portion, and which is protruded toward the recording section, and an upper end of the protruding portion is the highest in the contact portion.', ' CROSS REFERENCE TO RELATED APPLICATION The present application is a divisional application of prior U.S. application Ser. No. 13/629,937, filed Sep. 28, 2012, which claims priority from Japanese Patent Application No. 2011-259496, filed on Nov. 28, 2011, the disclosure of which is incorporated herein by reference in its entirety. BACKGROUND OF THE INVENTION 

/Users/patrickkennedy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


In [68]:
dumm = [w.lower().strip(string.punctuation) for w in row[6].split() if not w.isdigit()]
" ".join(dumm)

'cross reference to related application the present application is a divisional application of prior u.s application ser no 13/629,937 filed sep 28 2012 which claims priority from japanese patent application no 2011-259496 filed on nov 28 2011 the disclosure of which is incorporated herein by reference in its entirety background of the invention 1 field of the invention the present invention relates to an ink-jet recording apparatus which records an image on a sheet upon suppressing lifting-off of the sheet from a platen 2 description of the related art an ink-jet recording apparatus which includes a platen which supports a sheet transported in a transporting direction hereinafter transporting direction and which carries out image recording by jetting ink droplets from a recording section onto a sheet on the platen has hitherto been known in such type of ink-jet recording apparatuses there is an apparatus which includes a contact portion which suppresses or inhibits lifting off of a sh

In [57]:
def write_row(root):
    abstract = ""
    description = ""
    claims = []
    doc_number = ""
    date = ""
    publication_type = ""
    patent_length = ""
    title = ""

    for child in root:
        if child.tag == 'abstract':
            for c in child:
                try:
                    abstract += " " + c.text.strip()
                except:
                    pass

                for e in c:
                    try:
                        abstract += " " + e.text.strip()
                    except:
                        pass

                    for d in e:
                        try:
                            abstract += " " + d.text.strip()
                        except:
                            pass

                        for f in d:
                            try:
                                abstract += " " + f.text.strip()
                            except:
                                pass

                            for g in f:
                                try:
                                    abstract += " " + g.text.strip()
                                except:
                                    pass

                                for h in g:
                                    try:
                                        abstract += " " + h.text.strip()
                                    except:
                                        pass

                                    for j in h:
                                        try:
                                            abstract += " " + j.text.strip()
                                        except:
                                            pass
        elif child.tag == 'description':
            for c in child:
                try:
                    description += " " + c.text.strip()
                except:
                    pass

                for e in c:
                    try:
                        description += " " + e.text.strip()
                    except:
                        pass
        elif child.tag == 'claims':
            for c in child:
                # print c.tag, c.attrib
                for cs in c:
                    try:
                        holder = cs.text.strip()
                    except:
                        pass

                    for d in cs:
                        try:
                            holder += " " + d.text.strip()
                        except:
                            pass
                        
                        for g in d:
                            try:
                                holder += " " + g.text.strip()
                            except:
                                pass
                    try:    
                        claims.append(holder)
                    except:
                        pass


        for c in child:
            if c.tag == 'publication-reference':
                for e in c:
                    for a in e:
                        if a.tag == 'date':
                            try:
                                date = a.text.strip()
                            except:
                                pass
                        elif a.tag == 'doc-number':
                            try:
                                doc_number = a.text.strip()
                            except:
                                pass
                        elif a.tag == 'kind':
                            try:
                                publication_type = a.text.strip()
                            except:
                                pass
            if c.tag == 'us-term-of-grant':
                for e in c:
                    if e.tag == 'length-of-grant':
                        try:
                            patent_length = e.text.strip()
                        except:
                            pass

            elif c.tag == 'invention-title':
                try:
                    title = c.text.strip()
                except:
                    pass


    row = doc_number, date, publication_type, patent_length, \
            title, abstract, description, claims

    new_row = []
    claim_elem = []


    for elem in row[:-1]:
        new_elem = elem.encode('ascii', 'ignore')
        new_row.append(new_elem)    

    for elem in claims:
        new_elem = elem.encode('ascii', 'ignore')
        claim_elem.append(new_elem)
    new_row.append(claim_elem)

    return tuple(new_row)
        



In [54]:
data_as_string = "".join(data[1529])

root = ET.fromstring(data_as_string)

abstract = ""
    
for child in root:
    if child.tag == 'description':
        for c in child:
            try:
                description += "" + c.text.strip()
            except:
                pass
            
            for e in c:
                try:
                    description += " " + e.text.strip()
                except:
                    pass

CROSS-REFERENCE TO RELATED APPLICATIONS
The present application is a continuation application filed under 35 U.S.C. 111(a) claiming benefit under 35 U.S.C. 120 and 365(c) of PCT International Application No. PCT/JP2013/050991, filed on Jan. 18, 2013 and designating the U.S., which claims priority to Japanese Patent Application No. 2012-009438, filed on Jan. 19, 2012. The entire contents of all of the foregoing applications are incorporated herein by reference.
BACKGROUND OF THE INVENTION
1. Field of the Invention
The present invention relates to printers and methods of controlling a printer.
2. Description of the Related Art
Printers that output receipts are widely used for shop registers and automated teller machines (ATMs) or cash dispensers (CDs) in banks.
In such printers that output receipts, thermal paper serving as recording paper is wound in a roll, and printing is performed on the recording paper with a thermal head while conveying the recording paper. After conveying the reco

In [190]:
abstract = ""
description = ""
claims = []
doc_number = ""
date = ""
publication_type = ""
patent_length = ""
title = ""

for child in root:
    if child.tag == 'abstract':
        for c in child:
            try:
                abstract = c.text.strip()
            except:
                pass
            for e in c:
                try:
                    abstract += " " + e.text.strip()
                except:
                    pass
                
    elif child.tag == 'description':
        for c in child:
            try:
                description = c.text.strip()
            except:
                pass
    elif child.tag == 'claims':
        for c in child:
            # print c.tag, c.attrib
            for cs in c:
                try:
                    holder = cs.text.strip()
                except:
                    pass
                
                for d in cs:
                    try:
                        holder += " " + d.text.strip()
                    except:
                        pass
                try:    
                    claims.append(holder)
                except:
                    pass


    for c in child:
        if c.tag == 'publication-reference':
            for e in c:
                for a in e:
                    if a.tag == 'date':
                        try:
                            date = a.text.strip()
                        except:
                            pass
                    elif a.tag == 'doc-number':
                        try:
                            doc_number = a.text.strip()
                        except:
                            pass
                    elif a.tag == 'kind':
                        try:
                            publication_type = a.text.strip()
                        except:
                            pass
        if c.tag == 'us-term-of-grant':
            for e in c:
                if e.tag == 'length-of-grant':
                    try:
                        patent_length = e.text.strip()
                    except:
                        pass

        elif c.tag == 'invention-title':
            try:
                title = c.text.strip()
            except:
                pass


row = doc_number, date, publication_type, patent_length, \
        title, abstract, description, claims

new_row = []
claim_elem = []


for elem in row[:-1]:
    new_elem = elem.encode('ascii', 'ignore')
    new_row.append(new_elem)    

for elem in claims:
    new_elem = elem.encode('ascii', 'ignore')
    claim_elem.append(new_elem)
new_row.append(claim_elem)

print tuple(new_row)
        


 

('PP026873', '20160628', 'P2', '', 'Strawberry plant named Vivaldi', 'A new and distinct cultivar of Strawberry plant named Vivaldi, characterized by its compact and semi-upright plant habit; moderately vigorous to vigorous growth habit; uniform fruit ripening; medium-sized conical fruits that are glossy and bright red in color; pleasant fruit aroma and taste; excellent fruit postharvest longevity; and resistance to Botrytis cinerea Phytophthora cactorum.', 'The aforementioned photographs, following observations and measurements describe plants grown during the spring and summer in a glass-covered greenhouse in Eck en Wiel, The Netherlands and under cultural practices typical of commercial Strawberry production. During the production of the plants, day temperatures ranged from 12 C. to 23 C. and night temperatures ranged from 8 C. to 12 C. Plants were 2.5 months old when the photographs and description were taken. In the following description, color references are made to The Royal Hor

In [107]:
claims = []
for child in root:
    if child.tag == 'claims':
        for c in child:
            for cs in c:
                holder = cs.text.encode('utf-8').strip()
                for d in cs:
                    holder += " " + d.text.encode('utf-8').strip()
                claims.append(holder)

print claims

['1. A new and distinct cultivar of Lagerstroemia']


In [197]:
num = 3
dummy = [1, 2, 3]

dummy[:num]

[1, 2, 3]

In [16]:
for child in root:
    print "CHILD: ", child.tag, child.attrib
    for c in child:
        print "c in CHILD: ", c.tag, c.text
        for e in c:
            print "e in c: ", e.tag, e.text
            for a in e:
                print "a in e: ", a.tag, a.text
                for b in a:
                    print "b in a: ", b.tag, b.text
                    for d in b:
                        print "d in b: ", d.tag, d.text


CHILD:  us-bibliographic-data-grant {}
c in CHILD:  publication-reference 

e in c:  document-id 

a in e:  country US
a in e:  doc-number D0759941
a in e:  kind S1
a in e:  date 20160628
c in CHILD:  application-reference 

e in c:  document-id 

a in e:  country US
a in e:  doc-number 29506777
a in e:  date 20141020
c in CHILD:  us-application-series-code 29
c in CHILD:  priority-claims 

e in c:  priority-claim 

a in e:  country EM
a in e:  doc-number 002450080-0008
a in e:  date 20140418
c in CHILD:  us-term-of-grant 

e in c:  length-of-grant 14
e in c:  disclaimer 

a in e:  text This patent is subject to a terminal disclaimer.
c in CHILD:  classification-locarno 

e in c:  edition 10
e in c:  main-classification 0101
c in CHILD:  classification-national 

e in c:  country US
e in c:  main-classification D 1121
e in c:  further-classification  D1127
c in CHILD:  invention-title Chocolate-covered confection
c in CHILD:  us-references-cited 

e in c:  us-citation 

a in e:  patcit

In [102]:
#data_as_string = "".join(data[291])
root = ET.fromstring(data_as_string)


In [103]:
for child in root:
#     print child.tag, child.attrib
#     if child.tag == 'abstract':
#         for c in child:
#             print c.text
#     if child.tag == 'description':
#         for c in child:
#             print c.text
    if child.tag == 'claims':
        for c in child:
            print c.tag, c.attrib
            for cs in c:
                print cs.text

claim {'num': '00001', 'id': 'CLM-00001'}
The ornamental design for a chocolate-covered confection, as shown and described.


In [104]:
data_as_string

'<?xml version="1.0" encoding="UTF-8"?>\r\n<!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v45-2014-04-03.dtd" [ ]>\r\n<us-patent-grant lang="EN" dtd-version="v4.5 2014-04-03" file="USD0759941-20160628.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20160613" date-publ="20160628">\r\n<us-bibliographic-data-grant>\r\n<publication-reference>\r\n<document-id>\r\n<country>US</country>\r\n<doc-number>D0759941</doc-number>\r\n<kind>S1</kind>\r\n<date>20160628</date>\r\n</document-id>\r\n</publication-reference>\r\n<application-reference appl-type="design">\r\n<document-id>\r\n<country>US</country>\r\n<doc-number>29506777</doc-number>\r\n<date>20141020</date>\r\n</document-id>\r\n</application-reference>\r\n<us-application-series-code>29</us-application-series-code>\r\n<priority-claims>\r\n<priority-claim sequence="01" kind="regional">\r\n<country>EM</country>\r\n<doc-number>002450080-0008</doc-number>\r\n<date>20140418</date>\r\n</priority-claim>\r\n</priority-cla

In [105]:
root.findall('.')
print root.findall('./abstract')
print root.findall('./description')
print root.findall('./claims/claim')

[]
[<Element 'description' at 0x110528890>]
[<Element 'claim' at 0x110528d50>]


In [4]:
url = 'http://patents.reedtech.com/pgrbft.php'
u = urlopen(url)
try:
    html = u.read().decode('utf-8')
finally:
    u.close()

soup = BeautifulSoup(html)

In [11]:
%time
for link in soup.select('a'):
    if link.text[-4:] == '.zip':
        href = link.get('href')
        filename = href.rsplit('/', 1)[-1]
        href = urljoin(url, quote(href))
        try:
            urlretrieve(href, filename)
            zip_ref = zipfile.ZipFile(ZIP_PATH+'/'+filename, 'r')
            zip_ref.extractall(EXTRACT_DATA_PATH)
            zip_ref.close()
        except:
            print('failed to download') #write to csv if it failed
        break     

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [12]:
print filename

ipg160705.zip


In [13]:
os.remove(filename)
unzipped_fname = filename.split('.')[0] + '.xml'

print unzipped_fname

ipg160705.xml


In [16]:
start = time()
move_to_s3(unzipped_fname)
print (time()-start)/60./60.

In [17]:
os.remove(EXTRACT_DATA_PATH + unzipped_fname)

In [15]:
def move_to_s3(filename):
    '''
    DOCSTRING: move_to_s3

    Move .zip files to S3.  Print to csv: timestamp, filename, success(1/0)

    Returns: None
    '''

    tmp_dir = '../data/'
    tmp_fname = filename

    target_bucket = 'patentgrants'
    s3 = boto3.resource('s3')

    try:
        s3.Object(target_bucket, tmp_fname).put(Body=open(tmp_dir + tmp_fname, 'rb'))
    except Exception as e:
        print e
    finally:
        return

In [83]:

url = 'http://patents.reedtech.com/pgrbft.php'

def get_data(url):
    u = urlopen(url)
    try:
        html = u.read().decode('utf-8')
    finally:
        u.close()

    soup = BeautifulSoup(html)
    
    for link in soup.select('a'):
        if link.text[-4:] == '.zip':    
            href = link.get('href')
            filename = href.rsplit('/', 1)[-1]
            href = urljoin(url, quote(href))
            try:
                urlretrieve(href, filename)
                zip_ref = zipfile.ZipFile(ZIP_PATH+'/'+filename, 'r')
                zip_ref.extractall(EXTRACT_DATA_PATH)
                zip_ref.close()
            except:
                print('failed to download')
            
            return

In [160]:
counter = 0
data = open_many(1000)
# find a way to make each file go through this process
# and put into a csv



header = ('abstract', 'description', 'claims')

with open('parsed_data_test.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for i in xrange(len(data)):
        abstract = ""
        description = ""
        claims = []
        
        data_as_string = "".join(data[i])
        try:
            root = ET.fromstring(data_as_string)
        except:
            counter += 1
            break
            
        for child in root:
            if child.tag == 'abstract':
                for c in child:
                    abstract = c.text.encode('utf-8').strip()
                    print abstract
            if child.tag == 'description':
                for c in child:
                    description = c.text.strip()
            if child.tag == 'claims':
                for c in child:
                    # print c.tag, c.attrib
                    for cs in c:
                        claims.append(cs.text.strip())
        row = abstract, description, claims
        writer.writerow(row)


A new garden rose plant of the Miniature class which has abundant, yellow flowers and attractive foliage. This new and distinct variety has shown to be uniform and stable in the resulting generations from asexual propagation.
A new garden rose plant of the Miniature class which has abundant, yellow flowers and attractive foliage. This new and distinct variety has shown to be uniform and stable in the resulting generations from asexual propagation.
A new and distinct variety of rose with long lasting, novel orange-red flowers, and attractive foliage with very good disease resistance. It exhibits upright growth with abundant flowers. The new variety propagates well from cuttings and by grafting. This new and distinct variety has shown to be uniform and stable in the resulting generations from asexual propagation.
A new peach variety (
A new cultivar of
A new and distinct cultivar of Strawberry plant named ‘Vivaldi’, characterized by its compact and semi-upright plant habit; moderately vi

UnicodeEncodeError: 'ascii' codec can't encode character u'\xb0' in position 327: ordinal not in range(128)

In [146]:
for child in root:
    if child.tag == 'claims':
        for c in child:
            for cs in c:
                print cs.text
        

The ornamental design for a chocolate-covered confection, as shown and described.


In [157]:
row

('',
 '',
 ['The ornamental design for a chocolate-covered confection, as shown and described.'])

In [1]:
import re
import zipfile
import csv
import boto3
import io
import zipfile
import os
from datetime import datetime
from urllib import urlopen, urlretrieve, quote
from urlparse import urljoin
from bs4 import BeautifulSoup



ZIP_PATH = '../code'
EXTRACT_DATA_PATH = '../data'
DATA_PATH = '../data/ipg160628.xml'

url = 'http://patents.reedtech.com/pgrbft.php'





def move_to_s3(filename):
    '''
    DOCSTRING: move_to_s3

    Move .zip files to S3.  Print to csv: timestamp, filename, success(1/0)

    Returns: None
    '''

    tmp_dir = '../data/'
    tmp_fname = filename

    target_bucket = 'patentgrants'
    s3 = boto3.resource('s3')

    try:
        s3.Object(target_bucket, tmp_fname).put(Body=open(tmp_dir + tmp_fname, 'rb'))
    except Exception as e:
        print e
    finally:
        return

In [2]:
with open('get_data_logfile.csv', 'wb') as csvfile:
    logwriter = csv.writer(csvfile, delimiter=',')
    logwriter.writerow(['filename', 'success', 'timestamp'])



u = urlopen(url)
try:
    html = u.read().decode('utf-8')
finally:
    u.close()

soup = BeautifulSoup(html)

for link in soup.select('a'):
    if link.text[-4:] == '.zip':
        href = link.get('href')
        filename = href.rsplit('/', 1)[-1]
        href = urljoin(url, quote(href))
        try:
            urlretrieve(href, filename)
            zip_ref = zipfile.ZipFile(ZIP_PATH+'/'+filename, 'r')
            zip_ref.extractall(EXTRACT_DATA_PATH)
            zip_ref.close()

            

        except:
            print 'failed to download: {} @ {}'.format(filename,
                                                datetime.now())
        break





In [3]:
try:
    os.remove(filename)
    unzipped_fname = filename.split('.')[0] + '.xml'

    move_to_s3(unzipped_fname)

except:
    print 'failed to download: {} @ {}'.format(filename,
                                        datetime.now())

In [5]:
print EXTRACT_DATA_PATH
print unzipped_fname

../data
ipg160705.xml


In [6]:
try:
    os.remove(EXTRACT_DATA_PATH + '/' + unzipped_fname)
except:
    print 'failed to download: {} @ {}'.format(filename,
                                        datetime.now())   
    
    #write to csv if it succeeded


In [8]:
log = open('get_data_logfile.csv','a')


In [10]:
log.write(",".join([unzipped_fname, '1', str(datetime.now())]))
log.close()

In [12]:
for link in soup.select('a'):
    print link

<a href="index.php">Home</a>
<a href="patent-products.php">Patents</a>
<a href="pgyb.php">Grant Yellow Book (Single-Page TIFF Images)</a>
<a href="pgmpi.php">Grant (Multi-Page TIFF Images)</a>
<a href="pgrbfti.php">Grant Red Book (Full Text with Embedded Images)</a>
<a href="pgrbft.php">Grant Red Book (Full Text)</a>
<a href="pgrbbib.php">Grant Red Book (Bibliographic Text)</a>
<a href="pgocr.php">Grant OCR Text</a>
<a href="pgog.php">Official Gazettes</a>
<a href="payb.php">Application Yellow Book (Single-Page TIFF Images)</a>
<a href="pampi.php">Application (Multi-Page TIFF Images)</a>
<a href="parbfti.php">Application Red Book (Full Text with Embedded Images)</a>
<a href="parbft.php">Application Red Book (Full Text)</a>
<a href="parbbib.php">Application Red Book (Bibliographic Text)</a>
<a href="assignment.php">Assignment XML</a>
<a href="maintfee.php">Maintenance Fee Events</a>
<a href="classdata.php">Patent Classification Data</a>
<a href="ifwpet.php">Patent IFW Petition Decisions

In [200]:
import pandas as pd

In [201]:
df = pd.read_csv('parsed_data_test.csv')

In [202]:
df.shape

(17875, 8)

In [203]:
df

,doc_number,date,publication_type,patent_length,title,abstract,description,claims
0,D0759941,20160628,S1,14.0,Chocolate-covered confection,NaN,NaN,['The ornamental design for a chocolate-covere...
1,D0759942,20160628,S1,14.0,Bra,NaN,NaN,"['The ornamental design for a bra, as shown an..."
2,D0759943,20160628,S1,14.0,Pair of jeans with interchangeable pockets,NaN,NaN,['I claim the ornamental design for a pair of ...
3,D0759944,20160628,S1,15.0,Shirt,NaN,NaN,"['The ornamental design for a shirt, as shown ..."
4,D0759945,20160628,S1,14.0,Bird wings,NaN,NaN,"['The ornamental design for bird wings, as sho..."
5,D0759946,20160628,S1,14.0,Hat with pennant shaped pocket for receiving e...,NaN,NaN,['The ornamental design for a hat with pennant...
6,D0759947,20160628,S1,14.0,Hat with pockets for receiving eyeglasses stems,NaN,NaN,['The ornamental design for a hat with pockets...
7,D0759948,20160628,S1,14.0,Shoe,NaN,NaN,"['The ornamental design for a shoe, as shown a..."
8,D0759949,20160628,S1,15.0,Sandal,NaN,NaN,"['The ornamental design for a sandal, as shown..."
9,D0759950,20160628,S1,14.0,Footwear,NaN,NaN,"['The ornamental design for footwear, as shown..."


In [212]:
df.description.values[-1000]

'It will be appreciated that while a particular sequence of steps and materials has been shown and described for purposes of explanation, the sequence or materials may be varied in certain respects, or the steps/materials may be combined, while still obtaining the desired configuration. Additionally, modifications to the disclosed embodiment and the invention as claimed are possible and within the scope of this disclosed invention.'